In [20]:
import numpy as np

import sys
sys.path.append("..") 

from scipy.integrate import solve_ivp
from SciComp._ode_solvers import rk4_step, heun_step, euler_step


In [22]:
def scipy_solver(bvp, t):
    """
    Function to solve the PDE using Scipy's solve_ivp.

    Parameters
    ----------
    bvp : BVP object
        Boundary value problem to solve. Object instantiated in SciComp/bvp.py.
    t : numpy.ndarray
        Array of time values.

    Returns
    -------
    u : numpy.ndarray
        Solution to the PDE.
    """
    A, b, x_array = bvp.construct_matrix()
    
    u_boundary = bvp.f_fun(x_array, t[0])

    def PDE(t, u, D, A, b):
        return bvp.C * (A @ u + b) + bvp.q_fun(x_array, t, u)  

    sol = solve_ivp(PDE, (t[0], t[-1]), u_boundary, method='RK45', t_eval=t, args=(bvp.D, A, b))

    u = sol.y

    return u


def implicit_euler(bvp, t):
    """
    Function to solve the PDE using the implicit Euler method.

    Parameters
    ----------
    bvp : BVP object
        Boundary value problem to solve. Object instantiated in SciComp/bvp.py.
    t : numpy.ndarray
        Array of time values.
    source_fun : callable
        Function that computes the source term value at each grid point and time.

    Returns
    -------
    solution : numpy.ndarray
        Solution to the PDE.
    """
    A, b, x_array = bvp.construct_matrix()

    I = np.eye(bvp.shape)

    lhs = I - bvp.C * A

    u = np.zeros((len(x_array), len(t)))
    u[:, 0] = bvp.f_fun(x_array, t[0])

    for ti in range(0, len(t)-1):
        dt = t[ti+1] - t[ti]
        q = bvp.q_fun(x_array, t[ti+1], u[:, ti])
        u[:, ti+1] = np.linalg.solve(lhs, u[:, ti] + bvp.C * b + dt * q)

    return u


In [9]:
def imex_euler(bvp, t):
    """
    Function to solve the PDE using the IMEX Euler method.

    Parameters
    ----------
    bvp : BVP object
        Boundary value problem to solve. Object instantiated in SciComp/bvp.py.
    t : numpy.ndarray
        Array of time values.

    Returns
    -------
    solution : numpy.ndarray
        Solution to the PDE.
    """
    A, b, x_array = bvp.construct_matrix()

    I = np.eye(bvp.shape)

    # Construct left and right sides of the equation for the implicit part (diffusive terms)
    lhs = I - bvp.C * A
    rhs = bvp.C * b

    u = np.zeros((len(x_array), len(t)))
    u[:, 0] = bvp.f_fun(x_array, t[0])

    # Solve for each timestep
    for ti in range(0, len(t)-1):
        # Implicit part (diffusive terms)
        u[:, ti+1] = np.linalg.solve(lhs, u[:, ti] + rhs)

        # Explicit part (nonlinear terms)
        if bvp.q_fun is not None:
            u[:, ti+1] += bvp.q_fun(x_array, t[ti], u[:, ti])

    return u

In [25]:
import sys
sys.path.append("..") 

from SciComp.plotting import animate_PDE
from SciComp.bvp import BVP
import numpy as np

# Bratu problem
a = 0
b = 1
N = 20
alpha = 0
beta = 0
D = 1
mu = 2
q_fun = lambda x, t, u: np.exp(mu*u)
f_fun = lambda x, t: np.zeros(len(x))
bvp = BVP(a, b, N, alpha, beta, condition_type='Dirichlet', q_fun=q_fun, f_fun=f_fun, D=D)

t_boundary = 0
t_final = 1
dt = 0.001

t, dt, C = bvp.time_discretization(t_boundary, t_final, dt=dt, method='imex euler')

u1 = imex_euler(bvp, t)
u2 = implicit_euler(bvp, t)
u3 = scipy_solver(bvp, t)

# Check that the solutions are similar
print(np.allclose(u1, u2))
print(np.allclose(u2, u3))
print(np.allclose(u1, u3))


False


C:\Users\duppu\AppData\Local\Temp\ipykernel_24960\2821520491.py:16: RuntimeWarning: overflow encountered in exp
  q_fun = lambda x, t, u: np.exp(mu*u)
C:\Users\duppu\AppData\Local\Temp\ipykernel_24960\2623573734.py:22: RuntimeWarning: invalid value encountered in matmul
  return bvp.C * (A @ u + b) + bvp.q_fun(x_array, t, u)
C:\Users\duppu\AppData\Local\Temp\ipykernel_24960\2623573734.py:22: RuntimeWarning: invalid value encountered in add
  return bvp.C * (A @ u + b) + bvp.q_fun(x_array, t, u)


ValueError: operands could not be broadcast together with shapes (19,1001) (19,501) 

In [66]:
u[:, 100]

array([0.00000000e+00, 2.49922750e-04, 6.26656192e-02, 9.41076950e-02,
       1.25332738e-01, 1.56433848e-01, 1.87380575e-01, 2.18142380e-01,
       2.48688905e-01, 2.78990005e-01, 3.09015775e-01, 3.38736583e-01,
       3.68123100e-01, 3.97146323e-01, 4.25777611e-01, 4.53988708e-01,
       4.81751772e-01, 5.09039406e-01, 5.35824680e-01, 5.62081159e-01,
       5.87782932e-01, 6.12904634e-01, 6.37421474e-01, 6.61309255e-01,
       6.84544404e-01, 7.07103990e-01, 7.28965750e-01, 7.50108109e-01,
       7.70510201e-01, 7.90151893e-01, 8.09013801e-01, 8.27077309e-01,
       8.44324593e-01, 8.60738629e-01, 8.76303221e-01, 8.91003007e-01,
       9.04823481e-01, 9.17751003e-01, 9.29772816e-01, 9.40877055e-01,
       9.51052762e-01, 9.60289895e-01, 9.68579338e-01, 9.75912909e-01,
       9.82283373e-01, 9.87684442e-01, 9.92110785e-01, 9.95558035e-01,
       9.98022789e-01, 9.99502615e-01, 9.99996052e-01, 9.99502615e-01,
       9.98022789e-01, 9.95558035e-01, 9.92110785e-01, 9.87684442e-01,
      

In [42]:
A, b, x_array = bvp.construct_matrix()

u = np.zeros((len(x_array), len(t)))
u[:, 0] = bvp.f_fun(x_array, t[0])
u[0, :] = bvp.alpha
u[-1, :] = bvp.beta

def PDE(t, u):
   return bvp.C * (A @ u + b)

C:\Users\duppu\AppData\Local\Temp\ipykernel_18072\2717889184.py:8: RuntimeWarning:

divide by zero encountered in divide

C:\Users\duppu\AppData\Local\Temp\ipykernel_18072\2717889184.py:8: RuntimeWarning:

invalid value encountered in sin



In [48]:
]

0.0

In [158]:
print(u.shape)